Game Description:
https://yunhai.ctrip.com/Games/9

In [1]:
# initial environment
import pandas as pd
import numpy as np
import numpy.random as nr

nr.seed(42)

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
product_info = pd.read_csv("product_data/product_info.txt")
product_quantity = pd.read_csv("product_data/product_quantity.txt")

In [9]:
product_info.shape
product_info.columns

Index([u'product_id', u'district_id1', u'district_id2', u'district_id3',
       u'district_id4', u'lat', u'lon', u'railway', u'airport', u'citycenter',
       u'railway2', u'airport2', u'citycenter2', u'eval', u'eval2', u'eval3',
       u'eval4', u'voters', u'startdate', u'upgradedate', u'cooperatedate',
       u'maxstock'],
      dtype='object')

In [4]:
product_quantity.shape

(1959676, 9)

In [5]:
product_info.head()

,product_id,district_id1,district_id2,district_id3,district_id4,lat,lon,railway,airport,citycenter,...,citycenter2,eval,eval2,eval3,eval4,voters,startdate,upgradedate,cooperatedate,maxstock
0,1,10201,20502,31003,45760,3.994928,11.634630,-1.0,-1.0,-1.0,...,-1.0,3.0,5,2.0,3.1,1034.0,2005-11-01,2015-01-01,2013-07-02,75.0
1,2,10201,20502,31003,45760,3.995148,11.636258,-1.0,-1.0,2.3,...,-1.0,3.0,4,2.0,3.4,1707.0,2005-02-28,2011-01-01,2014-12-16,172.0
2,3,10201,20502,31003,45760,3.994291,11.631246,-1.0,-1.0,10.3,...,-1.0,3.0,4,2.5,3.6,1739.0,2007-03-01,2014-01-01,2014-07-02,188.0
3,4,10201,20502,31003,55952,3.997783,11.641561,-1.0,-1.0,9.3,...,-1.0,3.0,4,2.5,3.5,1065.0,2006-07-01,1753-01-01,2014-12-19,116.0
4,5,10201,20502,31003,55952,3.999904,11.641149,12.1,25.2,13.3,...,3.6,3.0,5,2.0,3.4,2209.0,2007-01-01,2012-03-01,2007-11-07,95.0


In [6]:
product_quantity.head()

,product_id,product_date,orderattribute1,orderattribute2,orderattribute3,orderattribute4,ciiquantity,ordquantity,price
0,1,2014-06-21,2,1,1,1,0,1,-1
1,1,2014-09-03,2,1,1,1,1,3,153
2,1,2014-09-26,2,1,1,1,0,3,-1
3,1,2014-11-28,2,1,1,1,0,4,-1
4,1,2014-01-11,2,1,1,1,1,1,153


In [10]:
1959676/30/4000

16

#### 1. 数据第一印象

4000个产品，1959676天的交易记录，每个产品大概有16个月每天每单的交易记录
1. 每个产品有22个属性, 21个有效属性。分为地理因素、评分、时间、库存4类。
2. 订单拥有9个属性，8个有用属性，分为时间、订单信息、预定数和实际交易数和价格。